In [1]:
import pandas as pd
import numpy as np
import pickle
import json
import sys
sys.path.append('.')

### Load Best Model

In [2]:
ctabgan_best = pickle.load(open('../research/best_models/ctabgan_best_model/pklmodel.pkl', 'rb'))

### Conditional Sampling

In [3]:
train_data = pd.read_csv('../thesisgan/input/new_train_data.csv')
hpo_data = pd.read_csv("../thesisgan/input/new_hpo_data.csv")
test_data = pd.read_csv("../thesisgan/input/new_test_data.csv")

In [4]:
og_syn_data = pd.read_csv("../research/best_models/ctabgan_best_model/syn.csv")

#### Get ML Utility Functions

In [5]:
from model.evaluation import get_utility_metrics
from sklearn.preprocessing import LabelEncoder
from model.data_preparation import DataPrep

In [6]:
data_prep = DataPrep(train_data, ['proto', 'tcp_ack', 'tcp_psh', 'tcp_rst', 'tcp_syn', 'tcp_fin', 'tos', 'label', 'attack_type'], 
                     [], {}, [], ['packets', 'src_pt', 'dst_pt', 'duration', 'bytes'], [],
                     {"Classification": 'attack_type'}, 0.00003)

In [7]:
more_samples = ctabgan_best.sample(train_data.shape[0])
og_more_samples = data_prep.inverse_prep(more_samples)

Sampled 274777 invalid samples. Have 890223 samples. Want 1164332 samples.
Sampled 64784 invalid samples. Have 1100439 samples. Want 1164332 samples.
Time taken: 14.450546741485596 seconds
Sampled 15189 invalid samples. Have 1150250 samples. Want 1164332 samples.
Time taken: 3.3146631717681885 seconds
Sampled 3763 invalid samples. Have 1162487 samples. Want 1164332 samples.
Time taken: 0.9744999408721924 seconds
Sampled 943 invalid samples. Have 1165544 samples. Want 1164332 samples.
Time taken: 0.22710895538330078 seconds


### Evaluate the Results

#### Original Results - train data, hpo data, fake data size of hpo data

In [8]:
import json
with open('../research/best_models/ctabgan_best_model/table/Classification Results_305_46aa8d602dd61325aed1.table.json') as f:
    og_cr = json.load(f)

In [9]:
le_dict = {"attack_type": "le_attack_type", "label": "le_label", "proto": "le_proto", "tos": "le_tos"}
for c in le_dict.keys():
    le_dict[c] = LabelEncoder()
    test_data[c] = le_dict[c].fit_transform(test_data[c])
    train_data[c] = le_dict[c].fit_transform(train_data[c])
    hpo_data[c] = le_dict[c].fit_transform(hpo_data[c])
    og_syn_data[c] = le_dict[c].fit_transform(og_syn_data[c])
    og_more_samples[c] = le_dict[c].fit_transform(og_more_samples[c])

In [13]:
og_cr = pd.DataFrame(data=og_cr["data"], columns=og_cr["columns"])

In [10]:
og_cr.drop(["Model"],axis=1).groupby(["Type"]).mean().sort_values(by="F1_Score", ascending=False).head(100)


,Acc,AUC,F1_Score
Type,,,
Real,78.035474,0.891350,0.584898
Fake,52.619821,0.832858,0.495669
Difference,25.415653,0.058492,0.089229


#### Original but more - train data, hpo data, fake data size of train data

In [26]:
more_samples_results, more_samples_cr = get_utility_metrics(train_data, hpo_data, og_more_samples, scaler="MinMax",type={"Classification":["xgb","lr","dt","rf","mlp"]})

Model:  xgb trained on real data
Model:  lr trained on real data
Model:  dt trained on real data
Model:  rf trained on real data
Model:  mlp trained on real data
Model:  xgb trained on fake data
Model:  lr trained on fake data
Model:  dt trained on fake data
